In [ ]:
# WORKS FOR GAMES FULL FULL STATS

In [91]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import traceback
import sqlite3 as sql

pd.set_option('display.max_columns', None)

In [93]:
%cd C:\Users\Joshua Broas\valorant-db

C:\Users\Joshua Broas\valorant-db


In [129]:
conn = sql.connect('valorant.db')

In [166]:
def cleanText(string):
    if string is not None:
        string = string.replace('\n','')
        string = string.replace('\t','')

    return string

def getMatchHeader(source, match_id):
    # Get Event Name and Date
    header = source.find_all("div", class_="match-header-super")[0]
    event = header.find("a", class_='match-header-event')
    a = header.find(href=True)
    event_url = a['href'].lstrip()
    event_id = event_url.split("/")[2]
    event_name = a.div.div.text.lstrip()
    event_stage = a.div.div.find_next_sibling('div').text.lstrip()
    date_header = header.find("div", class_="match-header-date")
    
    date_div = date_header.div
    date = date_div['data-utc-ts']
    
    # Get patch
    patch = None
    try:
        patch = date_header.div.find_next_sibling('div').find_next_sibling('div').text.lstrip()
    except:
        pass
    
    # Get Teams
    match_header_vs = source.find("div", class_="match-header-vs")
    a = match_header_vs.find_all(href=True)
    
    team1_id = a[0]['href'].split("/")[2]
    team2_id = a[1]['href'].split("/")[2]
    team1 = a[0].div.div.text.lstrip()
    team2 = a[1].div.div.text.lstrip()
    
    # Get Score
    match_score = source.find("div", class_="js-spoiler")
    team1_mapscore = match_score.span.text.lstrip()
    team2_mapscore = match_score.span.find_next_sibling('span').find_next_sibling('span').text.lstrip()
    
    # Clean text
    
    event_stage = cleanText(event_stage)
    event_name = cleanText(event_name)
    patch = cleanText(patch)
    team1 = cleanText(team1)
    team2 = cleanText(team2)
    team1_mapscore = cleanText(team1_mapscore)
    team2_mapscore = cleanText(team2_mapscore)
    
    res = {
        'MatchID': match_id,
        'Date': date,
        'Patch': patch,
        'EventID': event_id,
        'EventName': event_name,
        'EventStage': event_stage,
        'Team1ID': int(team1_id),
        'Team2ID': int(team2_id),
        'Team1': team1,
        'Team2': team2,
        'Team1_MapScore': int(team1_mapscore),
        'Team2_MapScore': int(team2_mapscore)
    }
    
    return res

In [167]:
def extractEconomyRoundByRound(cols):
    
    ecoDict = {
        '':'eco',
        '$':'semi-eco',
        '$$':'semi-buy',
        '$$$': 'full-buy'
    }
    
    ecoRoundByRound = {}
    
    for col in cols:
        check = col.find('div', class_='ge-text-light round-num')
        if check is not None:
            t1_buytype = cleanText(col.div.find_next_sibling('div').find_next_sibling('div').text)
            t2_buytype = cleanText(col.div.find_next_sibling('div').find_next_sibling('div').find_next_sibling('div').text)
            roundNum = cleanText(check.text)
            bank = col.find_all('div', class_='bank')
            bankt1 = cleanText(bank[0].text)
            bankt2 = cleanText(bank[1].text)
            
            bankt1 = float(bankt1[:-1])*1000
            bankt2 = float(bankt2[:-1])*1000
            
            
            ecoRoundByRound[int(roundNum)] = {
                'Team1Bank': bankt1,
                'Team2Bank': bankt2,
                'Team1BuyType': ecoDict[t1_buytype],
                'Team2BuyType': ecoDict[t2_buytype]
                }
    
    return ecoRoundByRound

def extractEconomyRows(row, game_id):
    pistolWon_t1 = cleanText(row[0].text)
    eco_list_t1 = cleanText(row[1].text).replace('(', ' ').replace(')',' ').split()
    semi_eco_list_t1 = cleanText(row[2].text).replace('(', ' ').replace(')',' ').split()
    semi_buy_list_t1 = cleanText(row[3].text).replace('(', ' ').replace(')',' ').split()
    full_buy_list_t1 = cleanText(row[4].text).replace('(', ' ').replace(')',' ').split()
    
    pistolWon_t2 = cleanText(row[5].text)
    eco_list_t2 = cleanText(row[6].text).replace('(', ' ').replace(')',' ').split()
    semi_eco_list_t2 = cleanText(row[7].text).replace('(', ' ').replace(')',' ').split()
    semi_buy_list_t2 = cleanText(row[8].text).replace('(', ' ').replace(')',' ').split()
    full_buy_list_t2 = cleanText(row[9].text).replace('(', ' ').replace(')',' ').split()
    
    res = {
        'GameID': game_id,
        'Team1_PistolWon': int(pistolWon_t1),
        'Team1_Eco': int(eco_list_t1[0]),
        'Team1_EcoWon': int(eco_list_t1[1]),
        'Team1_SemiEco': int(semi_eco_list_t1[0]),
        'Team1_SemiEcoWon': int(semi_eco_list_t1[1]),
        'Team1_SemiBuy': int(semi_buy_list_t1[0]),
        'Team1_SemiBuyWon': int(semi_buy_list_t1[1]),
        'Team1_FullBuy': int(full_buy_list_t1[0]),
        'Team1_FullBuyWon': int(full_buy_list_t1[1]),
        'Team2_PistolWon': int(pistolWon_t2),
        'Team2_Eco': int(eco_list_t2[0]),
        'Team2_EcoWon': int(eco_list_t2[1]),
        'Team2_SemiEco': int(semi_eco_list_t2[0]),
        'Team2_SemiEcoWon': int(semi_eco_list_t2[1]),
        'Team2_SemiBuy': int(semi_buy_list_t2[0]),
        'Team2_SemiBuyWon': int(semi_buy_list_t2[1]),
        'Team2_FullBuy': int(full_buy_list_t2[0]),
        'Team2_FullBuyWon': int(full_buy_list_t2[1]),
    }    
    return res

    
def getEconomy(url, game_id):
    full_url = url + '?game=' + game_id[0] + '&tab=economy'
    page = requests.get(full_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    economy_dict_list = []
    economy_rbr_dict_list = {}
    
    container = soup.find_all('table', class_='wf-table-inset mod-econ')

    idx_1=0
    idx_2=0
    for table in container:
        data_points = table.find_all('div', class_='stats-sq')
        if len(data_points) > 0 and idx_1 <= len(game_id)-1:
            economy_dict_list.append(extractEconomyRows(data_points, game_id[idx_1]))
            idx_1 = idx_1 + 1
        else:
            if idx_2 <= len(game_id)-1:
                extractTD = table.find_all('td')
                economy_rbr_dict_list[game_id[idx_2]] = extractEconomyRoundByRound(extractTD)
                idx_2 = idx_2 + 1
    
    return economy_dict_list, economy_rbr_dict_list

In [168]:
def checkIfEmpty(html):
    split = html.text.split()
    if len(split) == 0:
        return 0
    else:
        return int(split[0])
    

def extractRowPreformanceData(row, game_id):
    player_team = row[0].text.split()
    player = player_team[0]
    team = player_team[1]
    
    agent_pic = row[1].find('img')['src']
    agent = agent_pic.split('/')[5].replace('.png', '')
    
    num_2Ks = checkIfEmpty(row[2])
    num_3Ks = checkIfEmpty(row[3])
    num_4Ks = checkIfEmpty(row[4])
    num_5Ks = checkIfEmpty(row[5])
    OnevOne = checkIfEmpty(row[6])
    OnevTwo = checkIfEmpty(row[7])
    OnevThree = checkIfEmpty(row[8])
    OnevFour = checkIfEmpty(row[9])
    OnevFive = checkIfEmpty(row[10])
    econ = checkIfEmpty(row[11])
    plants = checkIfEmpty(row[12])
    defuses = checkIfEmpty(row[13])

    res = {
        'GameID': game_id,
        'PlayerName': player,
        'TeamAbbreviation': team,
        'Agent': agent,
        'Num_2Ks': num_2Ks,
        'Num_3Ks': num_3Ks,
        'Num_4Ks': num_4Ks,
        'Num_5Ks': num_5Ks,
        'OnevOne': OnevOne,
        'OnevTwo': OnevTwo,
        'OnevThree': OnevThree,
        'OnevFour': OnevFour,
        'OnevFive': OnevFive,
        'Econ': econ,
        'Plants': plants,
        'Defuses': defuses
    }
    
    return res


def getPerformanceData(url, game_id):
    full_url = url + '?game=' + game_id[0] + '&tab=performance'
    page = requests.get(full_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    containers_all = soup.find_all('table', class_='wf-table-inset mod-adv-stats')
    if len(containers_all) == 0:
        return None
    
    containers_all.pop(0)
    
    player_stats = []
    for idx, container in enumerate(containers_all):
        rows = container.find_all('tr')
        for row in rows:
            team = row.find('div', class_='team')
            if team is None:
                continue
            data_points = row.find_all('td')
            row_data = extractRowPreformanceData(data_points, game_id[idx])
            player_stats.append(row_data)
            
    return player_stats

In [188]:
def returnEmptyRow(row):
    name_team_list = cleanText(row[0].text).split()
    if len(name_team_list) == 2:
        player = name_team_list[0]
        team = name_team_list[1]
    else:
        player = name_team_list[0]
        team = ''
        
    res = {
        'GameID': None,
        'PlayerID': None,
        'PlayerName': player,
        'TeamAbbreviation': team,
        'Agent': None,
        'ACS': None,
        'Kills': None,
        'Deaths': None,
        'Assists': None,
        'PlusMinus': None,
        'KAST_Percent': None,
        'ADR': None,
        'HS_Percent': None,
        'FirstKills': None,
        'FirstDeaths': None,
        'FKFD_PlusMinus': None
    }
    
    return res

def extractRowData(row):
    name_team_list = cleanText(row[0].text).split()
    
    player = name_team_list[0]
    team = name_team_list[1]
    
    combat_score = cleanText(row[2].text)
    try:
        combat_score = int(combat_score)
    except:
        combat_score = None
    kills =  cleanText(row[3].text)
    try:
        kills = int(kills)
    except:
        kills = None
    deaths_unformatted =  cleanText(row[4].text) # need to clean slashes later
    assists =  cleanText(row[5].text)
    try:
        assists = int(assists)
    except:
        assists = None
    plus_minus_unformatted =  cleanText(row[6].text) # need to clean + later
    kast_unformatted =  cleanText(row[7].text)
    adr =  cleanText(row[8].text)
    try:
        adr = int(adr)
    except:
        adr = None
    hs_percent_unformatted =  cleanText(row[9].text)
    first_kills =  cleanText(row[10].text)
    try:
        first_kills = int(first_kills)
    except:
        first_kills = None
    first_deaths =  cleanText(row[11].text)
    try:
        first_deaths = int(first_deaths)
    except:
        first_deaths = None
    fkfd_plus_minus_unformatted = cleanText(row[12].text)
    
    
    # Now onto formatting some of the fields...
    deaths = deaths_unformatted.replace('/','')
    try:
        deaths = int(deaths)
    except:
        deaths = None
    plus_minus = plus_minus_unformatted.replace('+', '')
    try:
        plus_minus = int(plus_minus)
    except:
        plus_minus = None
    kast = kast_unformatted.replace('%','')
    try:
        kast = int(kast)/100 # convert % to decimal
    except:
        kast = None
    hs_percent = hs_percent_unformatted.replace('%','')
    try:
        hs_percent = int(hs_percent)/100 # convert % to decimal
    except:
         hs_percent = None
    fkfd_plus_minus = fkfd_plus_minus_unformatted.replace('+', '')
    try:
        fkfd_plus_minus = int(fkfd_plus_minus)
    except:
        fkfd_plus_minus = None
    
    res = {
        'GameID': None,
        'PlayerID': None,
        'PlayerName': player,
        'TeamAbbreviation': team,
        'Agent': None,
        'ACS': combat_score,
        'Kills': kills,
        'Deaths': deaths,
        'Assists': assists,
        'PlusMinus': plus_minus,
        'KAST_Percent': kast,
        'ADR': adr,
        'HS_Percent': hs_percent,
        'FirstKills': first_kills,
        'FirstDeaths': first_deaths,
        'FKFD_PlusMinus': fkfd_plus_minus
    }
    
    return res
    

def getScoreboard(url, game_id, team1_id, team2_id, match_id):
    full_url = url + '?game=' + game_id[0] + '&tab=overview'
    page = requests.get(full_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    def getScoreboardHeader(soup, match_id):
        # Scoreboard Header
        game_json_list = []
        stats_header = soup.find_all('div', class_='vm-stats-game-header')
        for idx, tab in enumerate(stats_header):
            team_list = tab.find_all('div', class_='team-name')
            team1 = cleanText(team_list[0].text)
            team2 = cleanText(team_list[1].text)    
            score_list = tab.find_all('div', class_='score')
            team1_score = int(cleanText(score_list[0].text))
            team2_score = int(cleanText(score_list[1].text))

            if team1_score > team2_score:
                winner = team1
            else:
                winner = team2

            half_score_list = tab.find_all('span')
            
            # Non-OT
            if len(half_score_list) <= 5:
                team1_firsthalf_score = half_score_list[0].text
                team1_secondhalf_score = half_score_list[1].text
                map_name = cleanText(half_score_list[2].text)
                team2_firsthalf_score = half_score_list[3].text
                team2_secondhalf_score = half_score_list[4].text

                if 'mod-ct' in half_score_list[0]['class']:
                    team1_firsthalf_side = 'defend'
                else:
                    team1_firsthalf_side = 'attack'

                if 'mod-ct' in half_score_list[3]['class']:
                    team2_firsthalf_side = 'defend'
                else:
                    team2_firsthalf_side = 'attack'
                
                team1_roundsOT = 0
                team2_roundsOT = 0
            
            # Matches with OT
            else:
                team1_firsthalf_score = half_score_list[0].text
                team1_secondhalf_score = half_score_list[1].text
                team1_roundsOT = int(half_score_list[2].text)
                map_name = cleanText(half_score_list[3].text)
                team2_firsthalf_score = half_score_list[4].text
                team2_secondhalf_score = half_score_list[5].text
                team2_roundsOT = int(half_score_list[6].text)

                if 'mod-ct' in half_score_list[0]['class']:
                    team1_firsthalf_side = 'defend'
                else:
                    team1_firsthalf_side = 'attack'

                if 'mod-ct' in half_score_list[4]['class']:
                    team2_firsthalf_side = 'defend'
                else:
                    team2_firsthalf_side = 'attack'

            duration_div = tab.find('div', class_='map-duration ge-text-light')
            duration = cleanText(duration_div.text)

            game_json = {
                'GameID': game_id[idx],
                'MatchID': match_id,
                'Map': map_name,
                'Team1ID': team1_id,
                'Team2ID': team2_id,
                'Team1': team1,
                'Team2': team2,
                'Winner': winner,
                'Team1_TotalRounds': team1_score,
                'Team2_TotalRounds': team2_score,
                'Team1_SideFirstHalf': team1_firsthalf_side,
                'Team2_SideFirstHalf': team2_firsthalf_side,
                'Team1_RoundsFirstHalf': int(team1_firsthalf_score),
                'Team1_RoundsSecondtHalf': int(team1_secondhalf_score),
                'Team1_RoundsOT': team1_roundsOT,
                'Team2_RoundsFirstHalf': int(team2_firsthalf_score),
                'Team2_RoundsSecondtHalf': int(team2_secondhalf_score),
                'Team2_RoundsOT': team2_roundsOT
            }
            
            game_json_list.append(game_json)

        return game_json_list
    
    def getRoundByRound(soup, game_id):
        # Round by round
        rounds_container_all = soup.find_all('div', class_='vlr-rounds-row')
        round_by_round_dict = {}
        
        for idx, rounds_container in enumerate(rounds_container_all):
            teams = rounds_container.find_all('div', class_='team')
            team1_abr = cleanText(teams[0].text)
            team2_abr = cleanText(teams[1].text)
            cols = rounds_container.find_all('div', class_='vlr-rounds-row-col')
            cols.pop(0)
            round_by_round = {}

            team1_start_side_flag = False
            team1_start_side = ''
            for col in cols:
                rnd_number = col.find('div', class_='rnd-num')
                if len(col['class']) == 1: #Length will be 2 with: 'vlr-rounds-row-col mod-spacing'
                    round_number = int(cleanText(rnd_number.text))
                    class_name = col.div.find_next_sibling('div')['class']
                    if 'mod-win' in class_name:
                        if team1_start_side_flag is False:
                            if 'mod-ct' in class_name:
                                team1_start_side = 'defend'
                                team1_start_side_flag = True
                            if 'mod-t' in class_name:
                                team1_start_side = 'attack'
                                team1_start_side_flag = True

                        win_type_img = col.find('img')
                        win_type = win_type_img['src'].split("/")[5].replace('.webp','')
                        curr_score_tag = col.find('div', class_='rnd-currscore')
                        current_score = cleanText(curr_score_tag.text)
                        round_by_round[round_number] = {
                            'RoundWinner': team1_abr,
                            'ScoreAfterRound': current_score,
                            'WinType': win_type,
                        }

                    else: # Team 2 won OR no rounds were played (game ended)
                        row2 = col.div.find_next_sibling('div').find_next_sibling('div')
                        row2_class = row2['class']
                        if 'mod-win' in row2_class:
                            win_type_img = row2.find('img')
                            win_type = win_type_img['src'].split("/")[5].replace('.webp','')
                            curr_score_tag = col.find('div', class_='rnd-currscore')
                            current_score = cleanText(curr_score_tag.text)
                            round_by_round[round_number] = {
                                'RoundWinner': team2_abr,
                                'ScoreAfterRound': current_score,
                                'WinType': win_type,
                            }

                        else:
                            continue
                            
                round_by_round_dict[game_id[idx]] = round_by_round
                
        return round_by_round_dict
    

    # Get Player Stats
    def getPlayerStats(soup):
        containers_all = soup.find_all('div', class_='vm-stats-game')
        
        checkIfNoStats = soup.find('div', class_='rnd-sq mod-win mod-ct')
        missingData = False
        if checkIfNoStats is None:
            missingData = True
        
        containers = []
        for check in containers_all: # Get rid of overall tab
            header = check.find('div', class_='vm-stats-game-header')
            if header is not None:
                containers.append(check)
            
        player_stats = []
        for idx, container in enumerate(containers):
            container = container.find_all('table', class_='wf-table-inset mod-overview')
            for table in container:
                rows = table.find_all('tr')
                for row in rows:
                    data_points = row.find_all('td')
                    agent_img = row.find('img')
                    player_id_search = row.find('a')

                    # There's times where it gets the headers which we don't want
                    if (len(data_points) > 0):
                        if missingData is True:
                            agent = ''
                            row_data = returnEmptyRow(data_points)
                        else:
                            try:
                                agent = agent_img['alt']
                            except:
                                agent = ''
                            row_data = extractRowData(data_points)
                            
                        player_id = player_id_search['href'].split('/')[2]
                        row_data['GameID'] = game_id[idx]
                        row_data['Agent'] = agent
                        row_data['PlayerID'] = player_id

                        player_stats.append(row_data)
        return player_stats
        
    try:
        game_dict = getScoreboardHeader(soup, match_id)
    except:
        print('Unable to execute getScoreboardHeader')
        traceback.print_exc()
    try:
        rbr_dict = getRoundByRound(soup, game_id)
    except:
        print('Unable to execute getRoundByRound')
        traceback.print_exc()
    try:
        player_stats_dict = getPlayerStats(soup)
    except:
        print('Unable to execute getPlayerStats')
        traceback.print_exc()
    
    return game_dict, rbr_dict, player_stats_dict

                
                
def getMapStats(source, url, team1_id, team2_id, match_id):
    header = source.find("div", class_="vm-stats")
    
    game_id = []
    no_select_div = header.find("div", class_="vm-stats-gamesnav-container")
    
    try:
        next_sibs = no_select_div.div.div.find_next_siblings('div')
        for tabs in next_sibs:
            if tabs.text.split()[1] != 'N/A':
                game_id.append(tabs['data-game-id'])
    except:
        # Need to handle case if its only a single game
        if len(game_id) == 0:
            game_id.append(header['data-game-id'])
    
    if len(game_id) == 0:
        return None, None, None, None
    
    game_dict_list, rbr_dict_list, player_stats_dict_list = getScoreboard(url, game_id, team1_id, team2_id, match_id)
    
    return game_dict_list, rbr_dict_list, player_stats_dict_list, game_id


In [ ]:
# Match table
#------------
# MatchID
# Date
# Patch
# EventID
# EventStage
# Team1ID
# Team2ID
# Team1
# Team2
# Team1_MapScore
# Team2_MapScore

# Game Table
# ------------
# GameID
# Map
# Duration
# Team1ID
# Team2ID
# Team1
# Team2
# Winner
# Team1_TotalRounds
# Team1_SideFirstHalf
# Team1_RoundsFirstHalf
# Team1_RoundsSecondtHalf
# Team1_PistolWon
# Team1_Eco
# Team1_EcoWon
# Team1_SemiEco
# Team1_SemiEcoWon
# Team1_SemiBuy
# Team1_SemiBuyWon
# Team1_FullBuy
# Team1_FullBuyWon
# Team2_TotalRounds
# Team2_SideFirstHalf
# Team2_RoundsFirstHalf
# Team2_RoundsSecondtHalf
# Team2_PistolWon
# Team2_Eco
# Team2_EcoWon
# Team2_SemiEco
# Team2_SemiEcoWon
# Team2_SemiBuy
# Team2_SemiBuyWon
# Team2_FullBuy
# Team2_FullBuyWon


# Game_Rounds
# ------------
# GameID
# Team1ID
# Team2ID
# RoundHistory      NOTE: contains dict of rounds

# format example: 
#    {1: {'RoundWinner': 'AKRE', 'ScoreAfterRound': '1-0', 'WinType': 'defuse', 'Team1Bank': 300.0, 'Team2Bank': 200.0,
#    'Team1BuyType': 'eco', 'Team2BuyType': 'eco'},
#    2: {'RoundWinner': 'AKRE', 'ScoreAfterRound': '2-0', 'WinType': 'elim', 'Team1Bank': 2200.0, 'Team2Bank': 10000.0,
#   'Team1BuyType': 'semi-buy', 'Team2BuyType': 'eco'}, ...


# Game_Scoreboard
# ------------
# GameID
# PlayerID
# TeamID
# PlayerName
# Agent
# Kills
# Deaths
# Assists
# PlusMinus
# KAST_Percent
# ADR
# HS_Percent
# FirstKills
# FirstDeaths
# FKFD_PlusMinus
# Num_2Ks
# Num_3Ks
# Num_4Ks
# Num_5Ks
# OnevOne
# OnevTwo
# OnevThree
# OnevFour
# OnevFive
# Econ
# Plants
# Defuses



In [189]:
# TEXT BOX
url = 'https://www.vlr.gg/191/prodigy-vs-fish123-cooler-cup-playoffs-qf'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

match_row = getMatchHeader(soup, 61393)
match_df = pd.DataFrame(match_row, index=[0])
team1_id = match_row['Team1ID']
team2_id = match_row['Team2ID']


game_dict_list, rbr_dict_list, player_stats_dict_list, game_id_list = getMapStats(soup, url, team1_id, team2_id)
print(game_id_list)

adv_player_stats = getPerformanceData(url, game_id_list)
player_stats_df = pd.DataFrame(player_stats_dict_list)
adv_player_stats_df = pd.DataFrame(adv_player_stats)

if adv_player_stats is not None:
    all_player_stats = player_stats_df.merge(adv_player_stats_df, on=['GameID', 'PlayerName', 'Agent'])
    all_player_stats = all_player_stats.drop(columns=['TeamAbbreviation_y'])
    all_player_stats = all_player_stats.rename(columns={'TeamAbbreviation_x':'TeamAbbreviation'})
else:
    all_player_stats = player_stats_df
    all_player_stats['Num_2Ks'] = None
    all_player_stats['Num_3Ks'] = None
    all_player_stats['Num_4Ks'] = None
    all_player_stats['Num_5Ks'] = None
    all_player_stats['OnevOne'] = None
    all_player_stats['OnevTwo'] = None
    all_player_stats['OnevThree'] = None
    all_player_stats['OnevFour'] = None
    all_player_stats['OnevFive'] = None
    all_player_stats['Econ'] = None
    all_player_stats['Plants'] = None
    all_player_stats['Defuses'] = None
    all_player_stats['Econ'] = None


economy_dict_list, economy_rbr_dict_list = getEconomy(url, game_id_list)

if len(economy_dict_list) > 0:
    game_df = pd.DataFrame(game_dict_list)
    econ_df = pd.DataFrame(economy_dict_list)
    full_game_df = game_df.merge(econ_df, on=['GameID'])

    game_rounds = []
    for key in rbr_dict_list:
        econ_info = economy_rbr_dict_list[key]
        for idx, round_num in enumerate(rbr_dict_list[key]):
            rbr_dict_list[key][round_num].update(econ_info[round_num])
            
        res = {
            'GameID': key,
            'Team1ID': team1_id,
            'Team2ID': team2_id,
            'RoundHistory': str(rbr_dict_list[key])
        }
        game_rounds.append(res)

    game_rounds_df = pd.DataFrame(game_rounds)
else:
    full_game_df = pd.DataFrame(game_dict_list)
    game_rounds = []
    for game in game_id_list:
        res = {
            'GameID': game,
            'Team1ID': team1_id,
            'Team2ID': team2_id,
            'RoundHistory': None
        }
        game_rounds.append(res)
    game_rounds_df = pd.DataFrame(game_rounds)
    
# match_df Match Table
# full_game_df Game Table
# game_rounds_df Game_Rounds Table
# all_player_stats Game_Scoreboard Table


['306']


In [190]:

try:
    match_df.to_sql('Matches', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Match Table')
    traceback.print_exc()
    pass
try:
    full_game_df.to_sql('Games', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Game Table')
    traceback.print_exc()
    pass
try:
    game_rounds_df.to_sql('Game_Rounds', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Game_Rounds Table')
    traceback.print_exc()
    pass
try:
    all_player_stats.to_sql('Game_Scoreboard', conn, index=False, if_exists='replace')
except:
    print('Unable to add to Game_Scoreboard Table')
    traceback.print_exc()
    pass

print('done!')

done!


In [210]:
try_list = [(61395, '/61395/bait-academy-vs-team-reckaroo-knights-monthly-gauntlet-december-ro16'), (61444, '/61444/god-particles-vs-reckoning-esports-asus-rog-showdown-series-sf'), (60388, '/60388/vietnamese-curry-vs-froggers-frag-valorant-major-fullerton-qf'), (60301, '/60301/booster-seat-gaming-vs-iwnl-frag-valorant-major-philadelphia-r1'), (55503, '/55503/ex-flickbaiters-vs-andbox-nerd-street-gamers-winter-championship-open-8-ro16'), (51266, '/51266/acend-vs-keyd-stars-valorant-champions-opening-a'), (57281, '/57281/samelol-vs-third-impact-vcs-midwest-showdown-qf'), (51367, '/51367/crimas-disciples-vs-nerdnecks-aen-spring-series-gf'), (51490, '/51490/portuguesa-vs-stars-horizon-copa-rakin-season-3-gf'), (52466, '/52466/evil-geniuses-vs-darkzero-esports-nerd-street-gamers-winter-championship-open-7-ro16'), (51715, '/51715/the-mafia-vs-betrayeds-fantasy-team-rival-1-qf'), (51013, '/51013/paper-rex-vs-alter-ego-the-esports-club-showdown-ubf'), (50794, '/50794/ivy-vs-natus-vincere-moscow-cybersport-series-4-main-event-gf'), (50150, '/50150/ucam-esports-club-vs-kova-icebox-northern-classic-gf'), (50637, '/50637/g2-gozen-vs-tenstar-nova-champions-tour-game-changers-series-iii-emea-ubqf'), (50648, '/50648/lapfap-vs-abb-aski-e-spor-white-champions-tour-game-changers-series-iii-emea-lr1'), (49477, '/49477/nrg-esports-vs-akrew-valorant-elite-showdown-ro16'), (49771, '/49771/paper-rex-vs-beast-royal-sea-challenge-group-b'), (49915, '/49915/louvre-vs-germmy-bleed-esports-community-tournament-main-event-sf'), (49422, '/49422/soniqs-vs-spongebob-squarepants-valorant-elite-showdown-group-stage-opening-j'), (49765, '/49765/boom-esports-vs-divisionx-gaming-royal-sea-challenge-group-a'), (49756, '/49756/boom-esports-vs-sharper-esport-royal-sea-challenge-group-a'), (47470, '/47470/last-dance-vs-team-vitality-valorant-open-tour-france-fall-qualifier-gf'), (48069, '/48069/havan-liberty-female-vs-stars-horizon-v-nus-protocolo-evolu-o-ubqf'), (49124, '/49124/funkboings-vs-pass-gaming-latam-oneshot-master-series-valorant-sf'), (48848, '/48848/rix-gg-thunder-vs-ivy-icebox-northern-classic-decider-a'), (27819, '/27819/renatus-white-vs-nigma-galaxy-female-champions-tour-game-changers-sea-fsl-elite-playoffs-lr3'), (43769, '/43769/rise-vs-cloud9-blue-champions-tour-north-america-last-chance-qualifier-ubsf'), (46983, '/46983/cloud9-academy-vs-yfp-gaming-nerd-street-gamers-winter-championship-open-2-ro16'), (47201, '/47201/enigma-gaming-vs-team-xo-penta-challenge-gf'), (46307, '/46307/flickbaiters-vs-shopify-rebellion-boomtv-proving-grounds-s3-semi-pro-division-r2'), (46306, '/46306/team-basilisk-vs-24haven-boomtv-proving-grounds-s3-semi-pro-division-r2'), (46701, '/46701/big-bang-vs-utage-season-3-main-event-sf'), (42182, '/42182/havan-liberty-vs-six-karma-champions-tour-south-america-last-chance-qualifier-decider-b'), (42175, '/42175/sharks-esports-vs-australs-champions-tour-south-america-last-chance-qualifier-winners-a'), (39490, '/39490/global-esports-vs-damwon-gaming-champions-tour-asia-pacific-last-chance-qualifier-ubro1'), (39489, '/39489/boom-esports-vs-fennel-champions-tour-asia-pacific-last-chance-qualifier-ubro1'), (43854, '/43854/team-eastern-europe-vs-team-italy-blast-spike-nations-2-sf'), (43997, '/43997/natus-vincere-vs-gmt-esports-meta-open-valorant-winners-a'), (43134, '/43134/shopify-rebellion-vs-dignitas-champions-tour-game-changers-series-iii-north-america-lbf'), (42549, '/42549/team-xo-vs-f1ls-esports-the-esports-club-gauntlet-season-2-lbf'), (35280, '/35280/zeal-esports-vs-the-infinity-esports-kjc-esports-valorant-invitational-group-stage-winners-a'), (42596, '/42596/angry-titans-vs-activat3d-esports-lvp-rising-series-4-ro16'), (42189, '/42189/zeta-gaming-vs-wygers-circuito-tormenta-liga-radiante-s1-regular-season'), (40473, '/40473/fox-gaming-vs-piercer-esports-strike-arabia-championship-finals-ubsf'), (40469, '/40469/fox-gaming-vs-geekay-esports-strike-arabia-championship-finals-ubqf'), (33853, '/33853/wildcard-gaming-vs-team-bliss-oceania-tour-championship-ubsf'), (34813, '/34813/kafalar-esports-vs-stanbul-wildcats-esa-open-fire-all-stars-playoffs-qf'), (35238, '/35238/axibyte-esports-vs-123tap-rix-gg-open-august-sf'), (35039, '/35039/o2-gaming-vs-divine-force-gaming-fgc-valorant-invitational-r4-low'), (34734, '/34734/123tap-vs-les-zinzins-de-lespace-lvp-rising-series-3-ro32'), (33320, '/33320/divize-vs-resonate-nerd-street-gamers-summer-championship-elim-b'), (31291, '/31291/kr-esports-vs-six-karma-champions-tour-latam-stage-3-challengers-playoffs-usf'), (30765, '/30765/guild-esports-vs-giants-gaming-champions-tour-stage-3-emea-challengers-playoffs-lr2'), (30761, '/30761/funplus-phoenix-vs-forze-champions-tour-stage-3-emea-challengers-playoffs-lr1'), (30754, '/30754/supermassive-blaze-vs-funplus-phoenix-champions-tour-stage-3-emea-challengers-playoffs-ubqf'), (30755, '/30755/g2-esports-vs-oxygen-esports-champions-tour-stage-3-emea-challengers-playoffs-ubqf'), (29745, '/29745/x10-esports-vs-galaxy-racer-champions-tour-sea-stage-3-challengers-playoffs-group-stage-group-c'), (29858, '/29858/edward-gaming-vs-lizhi-energy-vision-playoffs-lr2'), (25967, '/25967/ego-vs-yamagan-champions-tour-vietnam-stage-3-challengers-3-main-event-gf'), (31133, '/31133/wave-esports-vs-cgn-esports-kingdom-calling-3-tournament-sf'), (29251, '/29251/gen-g-vs-tsm-champions-tour-north-america-stage-3-challengers-2-ubqf'), (28931, '/28931/besiktas-esports-vs-futbolist-champions-tour-turkey-stage-3-challengers-2-lr2'), (29704, '/29704/havan-liberty-vs-onlyfans-champions-tour-brazil-stage-3-challengers-3-qf'), (28944, '/28944/gmt-esports-vs-ag-champions-tour-cis-stage-3-challengers-2-lr1'), (29445, '/29445/evil-geniuses-vs-team-basilisk-nerd-street-gamers-summer-championship-july-qualifier-ro16'), (29443, '/29443/divize-vs-last-round-save-nerd-street-gamers-summer-championship-july-qualifier-ro16'), (25988, '/25988/cerberus-esports-vs-divisionx-gaming-champions-tour-vietnam-stage-3-challengers-2-main-event-lbf'), (28564, '/28564/bigetron-astro-vs-alter-ego-champions-tour-indonesia-stage-3-challengers-2-main-event-sf'), (28333, '/28333/darkzero-esports-vs-knights-champions-tour-north-america-stage-3-challengers-2-qualifier-ro16'), (28334, '/28334/andbox-vs-tsm-champions-tour-north-america-stage-3-challengers-2-qualifier-ro16'), (28299, '/28299/t1-vs-taipan-esports-champions-tour-north-america-stage-3-challengers-2-qualifier-ro64'), (28305, '/28305/built-by-gamers-vs-carpe-noctem-champions-tour-north-america-stage-3-challengers-2-qualifier-ro6'), (28285, '/28285/ez5-vs-whats-going-on-champions-tour-north-america-stage-3-challengers-2-qualifier-ro64'), (28230, '/28230/cloud9-white-vs-evolved-gamers-champions-tour-north-america-stage-3-challengers-2-qualifier-ro1'), (28195, '/28195/wave-esports-vs-team-queso-champions-tour-europe-stage-3-challengers-2-uro16'), (28174, '/28174/team-queso-vs-yendez-champions-tour-europe-stage-3-challengers-2-ro32'), (28144, '/28144/19esports-vs-superior-champions-tour-europe-stage-3-challengers-2-ro64'), (27676, '/27676/motion-team-vs-velox-champions-tour-latam-stage-3-challengers-2-ubqf'), (27968, '/27968/freya-vs-the-goose-house-lunar-champions-tour-europe-stage-3-challengers-2-ro256'), (26678, '/26678/yfp-gaming-vs-1337-nerd-street-gamers-summer-championship-open-11-ro16'), (25576, '/25576/yalla-esports-vs-archnemesis-strike-arabia-championship-season-2-gf'), (24164, '/24164/infinity-esports-vs-akave-esports-champions-tour-latam-stage-3-challengers-1-ubsf'), (25531, '/25531/edward-gaming-vs-lizhi-imbatv-fearless-invitational-cup-group-a'), (24956, '/24956/team-liquid-vs-team-bds-champions-tour-europe-stage-3-challengers-1-lr1'), (24949, '/24949/team-liquid-vs-fnatic-champions-tour-europe-stage-3-challengers-1-uqf'), (24978, '/24978/oxygen-esports-vs-fire-flux-esports-champions-tour-turkey-stage-3-challengers-1-ubqf'), (23085, '/23085/team-bds-vs-megastitut-valorant-open-tour-france-summer-qualifier-gf'), (24023, '/24023/100-thieves-vs-noble-champions-tour-north-america-stage-3-challengers-1-ro16'), (24013, '/24013/luminosity-vs-complexity-champions-tour-north-america-stage-3-challengers-1-ro32'), (23922, '/23922/darkzero-esports-vs-night-raid-champions-tour-north-america-stage-3-challengers-1-ro128'), (23948, '/23948/andbox-vs-pirate-nation-champions-tour-north-america-stage-3-challengers-1-ro128'), (24120, '/24120/team-singularity-vs-aspire-champions-tour-cis-stage-3-challengers-1-uro16'), (23879, '/23879/team-finest-vs-og-champions-tour-europe-stage-3-challengers-1-ro32'), (23716, '/23716/19esports-vs-storm7-champions-tour-europe-stage-3-challengers-1-ro256'), (23740, '/23740/g2-esports-vs-la-menace-fant-me-champions-tour-europe-stage-3-challengers-1-ro256'), (22485, '/22485/heet-vs-wild-multigaming-magyar-esport-kupa-2-gf'), (22980, '/22980/tenstar-vs-area-valorant-uk-i-skirmish-gf'), (23081, '/23081/megastitut-vs-edelweiss-esports-valorant-open-tour-france-summer-qualifier-ubqf'), (23273, '/23273/kg-network-vs-endgame-ultimasters-aoc-2-main-event-sf'), (23338, '/23338/ag-vs-368-moscow-cybersport-series-superfinal-qf'), (22313, '/22313/zeal-esports-vs-paper-rex-kjc-esports-valorant-tournament-one-playoffs-grand-finals'), (22712, '/22712/gamelanders-blue-vs-furia-copa-rakin-season-1-gf'), (22689, '/22689/bren-esports-vs-galaxy-racer-steelseries-prime-tournament-philippines-gf'), (20681, '/20681/order-vs-wildcard-gaming-aoc-rising-heroes-anz-2021-gf'), (19820, '/19820/straight-outta-the-zoo-vs-mindfreak-oceania-tour-stage-02-ubqf'), (19074, '/19074/team-vikings-vs-x10-esports-valorant-champions-tour-stage-2-masters-reykjavik-main-event-ubqf'), (18335, '/18335/built-by-gamers-vs-sum-2-prove-nerd-street-gamers-summer-championship-may-qualifier-qf'), (19121, '/19121/megastitut-vs-excel-valorant-open-tour-france-spring-qualifier-gf'), (18654, '/18654/mcbtsmac-vs-paradox-esports-oceania-tour-stage-02-closed-qual-2-ubsf'), (18653, '/18653/mindfreak-vs-nebula-oceania-tour-stage-02-closed-qual-2-ubsf'), (16846, '/16846/ex-b4-vs-pain-gaming-all-stars-cup-season-1-tournament-gf'), (18631, '/18631/purpure-vs-gmt-esports-tusovka-cup-tournament-gf'), (17502, '/17502/renegades-vs-eunited-nerd-street-gamers-summer-championship-april-qualifier-qf'), (17505, '/17505/soniqs-vs-kooky-koalas-nerd-street-gamers-summer-championship-april-qualifier-qf'), (17522, '/17522/9z-team-vs-atropelados-champions-tour-latam-stage-2-challengers-finals-ubsf'), (17045, '/17045/havan-liberty-vs-ingaming-champions-tour-brazil-stage-2-challengers-3-main-event-winners-b'), (17349, '/17349/6045pirates-esports-vs-hashira-champions-tour-philippines-stage-2-challengers-3-main-event-gf'), (16992, '/16992/t1-vs-sentinels-champions-tour-north-america-stage-2-challengers-2-lbr1'), (16639, '/16639/order-vs-mindfreak-order-nova-invitational-gf'), (16479, '/16479/team-bds-vs-ninjas-in-pyjamas-champions-tour-europe-stage-2-challengers-2-qf'), (14153, '/14153/cloud9-white-vs-clg-red-challengers-tour-game-changers-series-i-north-america-main-event-gf'), (14457, '/14457/team-lightning-vs-fox-gaming-mge-north-africa-gf'), (14047, '/14047/furious-gaming-vs-meta-gaming-aorus-league-las-2-decider-a'), (13056, '/13056/tilted-vs-mindfreak-lpl-summer-cup-legends-2021-r5'), (13051, '/13051/tilted-vs-paradox-esports-lpl-summer-cup-legends-2021-r4'), (13269, '/13269/made-in-thailand-vs-sharper-esports-champions-tour-thailand-stage-1-challengers-3-main-event-sf'), (13043, '/13043/agg-vs-tilted-lpl-summer-cup-legends-2021-r3'), (11474, '/11474/faze-clan-vs-100-thieves-champions-tour-north-america-stage-1-challengers-3-main-event-gf'), (11657, '/11657/giants-gaming-vs-57-champions-tour-europe-stage-1-challengers-3-ro64'), (12077, '/12077/slick-vs-b4-esports-champions-tour-brazil-stage-1-challengers-3-elimination-b'), (11442, '/11442/equinox-esports-vs-on-the-way-champions-tour-north-america-stage-1-challengers-3-ro32'), (11939, '/11939/akave-esports-vs-pro-hub-gaming-champions-tour-latam-stage-1-challengers-3-main-event-lan-lbf'), (10258, '/10258/envy-vs-sentinels-champions-tour-north-america-stage-1-challengers-2-gf'), (13035, '/13035/tilted-vs-mcteam-lpl-summer-cup-legends-2021-r1'), (10455, '/10455/todak-vs-team-shift-efire-master-league-1-asia-group-stage-group-a'), (10312, '/10312/demon-vs-overpalastrike-oceania-tour-stage-1-closed-qual-1-qf'), (10688, '/10688/the-better-lg-vs-project-xy-pittsburgh-knights-monthly-gauntlet-february-gf'), (10304, '/10304/crazy-raccoon-vs-fav-gaming-technoblood-climber-summit-gf'), (10181, '/10181/cloud9-blue-vs-stay-small-stay-second-champions-tour-north-america-stage-1-challengers-2-ro128'), (10267, '/10267/vorax-fusion-vs-dunking-crazy-champions-tour-brazil-stage-1-challengers-2-open-qual-ro16'), (9994, '/9994/tubeple-gaming-vs-fatality-champions-tour-korea-stage-1-challengers-2-open-qual-ro16'), (9424, '/9424/slick-vs-vorax-fusion-champions-tour-brazil-stage-1-challengers-1-main-event-decider-a'), (9289, '/9289/immortals-vs-sentinels-champions-tour-north-america-stage-1-challengers-1-gf'), (9571, '/9571/p-ek-gaming-vs-no-org-champions-tour-latam-stage-1-challengers-2-open-qual-lan-ro16'), (9250, '/9250/lazer-klan-vs-pro-hub-gaming-valorant-challengers-stage-1-latam-main-event-lan-upper-final-b'), (9103, '/9103/vorax-fusion-vs-3p-valorant-challengers-stage-1-brazil-open-qual-round-of-16'), (8888, '/8888/nuturn-vs-bearclaw-gaming-douyu-bie-zhuang-cup-winter-gf'), (8961, '/8961/lazer-klan-vs-janus-licht-esports-valorant-challengers-stage-1-latam-open-qualifier-lan-qf'), (8390, '/8390/dignitas-vs-andbox-nerd-street-gamers-winter-championship-groups-winners-a'), (7515, '/7515/bren-esports-vs-x10-esports-sea-fire-championship-2020-playoffs-gf'), (8261, '/8261/boom-esports-vs-morph-team-easter-egg-invitational-gf'), (8101, '/8101/foxy-araikordai-vs-team-smg-ae-league-conqueror-sf'), (7810, '/7810/need-more-dm-vs-ipon-team-horizon-reapers-magyar-esport-kupa-gf'), (7763, '/7763/arctic-gaming-vs-eazy-one-gaming-valorant-fast-sf'), (7753, '/7753/todak-vs-innervoice-onslaught-valorant-league-playoffs-uf'), (7839, '/7839/team-lz-vs-tbd-wcs20-sea-community-tournament-playoffs-lbr3'), (7835, '/7835/7sins-vs-tbd-wcs20-sea-community-tournament-playoffs-lbr2'), (7531, '/7531/majstersztyk-vs-ne-no-i-ky-aim-lab-eu-series-5-round-of-16'), (7166, '/7166/bbl-esports-vs-futbolist-first-strike-turkey-main-event-gf'), (6870, '/6870/g2-esports-vs-orgless-first-strike-europe-main-event-qf'), (7113, '/7113/reckless-lads-vs-nerd-banana-first-strike-indonesia-main-event-ro16'), (5292, '/5292/rvn-vs-the-ultimates-first-strike-mena-gcc-iraq-qualifiers-gf'), (6043, '/6043/ng-black-vs-nxl-esl-sea-community-clash-lr1'), (4996, '/4996/anubis-gaming-vs-nasr-esports-first-strike-mena-levant-egypt-qualifiers-gf'), (5413, '/5413/fox-gaming-vs-team-majesty-first-strike-mena-north-africa-qualifiers-gf'), (6480, '/6480/exo-clan-vs-mindfreak-first-strike-oceania-group-stage-winners-a'), (6469, '/6469/prodigy-vs-linstitut-first-strike-europe-c-d-playoffs-qf'), (6432, '/6432/spacestation-gaming-vs-cloud9-blue-first-strike-north-america-umg-closed-qual-decider-a'), (6002, '/6002/havan-liberty-vs-team-vikings-aorus-league-br-2-lower-final-a'), (5864, '/5864/exile-esports-vs-project-eversio-first-strike-europe-qualifier-c-ro64'), (4177, '/4177/exo-clan-vs-order-chiefs-men-expert-invitational-gf'), (4565, '/4565/nrg-esports-vs-team-thrifty-first-strike-north-america-nsg-open-1-ro128'), (3653, '/3653/cr4zy-vs-lvlup-show-your-valor-tournament-group-stage'), (3514, '/3514/vision-strikers-vs-t1-korea-a-w-extreme-masters-asia-invitational-gf'), (4005, '/4005/majstersztyk-vs-entropiq-polish-esport-cup-2020-sf'), (3883, '/3883/andbox-vs-moon-raccoons-renegades-x-nsg-invitational-qual-2-gf'), (3794, '/3794/china-nguyen-vs-mamba-mode-gaming-30bomb-invitational-series-2-tournament-gf'), (3816, '/3816/sumn-fc-vs-oxygen-esports-epic31-online-gf'), (3858, '/3858/beastcoast-vs-spacestation-gaming-renegades-x-nsg-invitational-qual-1-gf'), (3118, '/3118/sumn-fc-vs-zarplata-glx-elite-series-group-d-gf'), (3112, '/3112/bbl-esports-vs-nolpenki-glx-elite-series-group-c-gf'), (3511, '/3511/noorg2-0-vs-gamelanders-copa-rakin-2-tournament-grand-final'), (3106, '/3106/forze-vs-rix-gg-glx-elite-series-group-b-gf'), (3100, '/3100/rox-team-vs-eloquence-glx-elite-series-group-a-gf'), (3142, '/3142/vimdolol-vs-gamelanders-evolution-open-brazil-tournament-grand-final'), (3167, '/3167/china-nguyen-vs-luminosity-nerd-street-gamers-september-monthly-gf'), (3165, '/3165/china-nguyen-vs-team-fyb-nerd-street-gamers-september-monthly-sf'), (3230, '/3230/forze-vs-be-logic-aim-lab-community-cup-gf'), (3166, '/3166/luminosity-vs-team-2d-nerd-street-gamers-september-monthly-sf'), (3151, '/3151/purple-cobras-vs-g2g-esport-rix-gg-cup-gf'), (3213, '/3213/vimdolol-vs-noorg2-0-gamers-club-ultimate-ii-main-event-semifinals'), (3161, '/3161/spot-up-vs-china-nguyen-nerd-street-gamers-september-monthly-qf'), (3162, '/3162/team-fyb-vs-renegades-nerd-street-gamers-september-monthly-qf'), (3163, '/3163/luminosity-vs-mystify-nerd-street-gamers-september-monthly-qf'), (3164, '/3164/rise-vs-team-2d-nerd-street-gamers-september-monthly-qf'), (3027, '/3027/opportunists-vs-feafeafea-occitanie-esports-2020-gf'), (3141, '/3141/oxygen-esports-vs-lol-intel-gaming-fest-2020-tournament-final'), (3152, '/3152/str99-vs-g2g-esport-rix-gg-cup-lbf'), (3217, '/3217/vision-strikers-vs-royal-never-give-up-douyu-bie-zhuang-cup-season-1-playoffs-gf'), (3149, '/3149/str99-vs-purple-cobras-rix-gg-cup-ubf'), (3002, '/3002/velocity-gaming-vs-villainous-india-today-league-valorant-cup-tournament-gf'), (3150, '/3150/g2g-esport-vs-zarplata-rix-gg-cup-lr5'), (2926, '/2926/vision-strikers-vs-attack-all-around-epulze-royal-sea-cup-tournament-gf'), (2792, '/2792/mindfreak-vs-akatsuki-lpl-challenger-season-3-tournament-uqf'), (2791, '/2791/exo-clan-vs-pants-down-lpl-challenger-season-3-tournament-uqf'), (2925, '/2925/mith-attitude-vs-attack-all-around-epulze-royal-sea-cup-tournament-lbf'), (2924, '/2924/vision-strikers-vs-mith-attitude-epulze-royal-sea-cup-tournament-ubf'), (3170, '/3170/b4-esports-vs-noorg2-0-gamers-club-ultimate-ii-main-event-lbf-b'), (2872, '/2872/exo-clan-vs-team-launch-rise-of-valour-finals-gf'), (2953, '/2953/entropiq-vs-totem-providence-tourstat-summer-clash-w3'), (2151, '/2151/reject-vs-absolute-jupiter-a-w-extreme-masters-pro-invitational-tournament-gf'), (2632, '/2632/mamba-mode-gaming-vs-luminosity-pulse-series-gf'), (2412, '/2412/g2-esports-vs-funplus-phoenix-blast-twitch-invitational-tournament-gf'), (2174, '/2174/totem-providence-vs-team666-nicecactus-summer-trophy-main-event-quarterfinals'), (1898, '/1898/gen-g-vs-team-solomid-pittsburgh-knights-invitational-gauntlet-series-gf'), (1635, '/1635/team-solomid-vs-sentinels-faze-clan-invitational-playoffs-gf'), (2820, '/2820/velocity-gaming-vs-godsquad-aorus-south-asia-cup-grand-final'), (1202, '/1202/sentinels-vs-team-solomid-30bomb-summer-cup-gf'), (1517, '/1517/sentinels-vs-t1-academy-30bomb-summer-cup-group-b'), (1456, '/1456/bonk-vs-9xxx-mandatory-gg-cup-qf'), (1486, '/1486/fish123-vs-esoteric-mandatory-gg-cup-ro32'), (1503, '/1503/ninjas-in-pyjamas-vs-aera-mandatory-gg-cup-ro64'), (1492, '/1492/opportunists-vs-whatthe-mandatory-gg-cup-ro128'), (1506, '/1506/rix-gg-vs-exodia5-mandatory-gg-cup-ro128'), (1470, '/1470/wave-esports-vs-goodbye-myteq-legend-series-5-ubf'), (1314, '/1314/way-2-french-vs-echo-8-pax-arena-invitational-tournament-group-b'), (1264, '/1264/morning-light-vs-mamba-mode-gaming-nerd-st-gamers-open-15-tournament-gf'), (1147, '/1147/way-2-french-vs-team-underrated-fight-night-arena-8-gf'), (1136, '/1136/opportunists-vs-nolpenki-weplay-valorant-invitational-qualifier-2-gf'), (1131, '/1131/need-more-dm-vs-team-xz-weplay-valorant-invitational-qualifier-1-gf'), (1089, '/1089/g2-esports-vs-prodigy-vitality-european-open-playoffs-gf'), (1088, '/1088/prodigy-vs-fabriken-vitality-european-open-playoffs-lbf'), (1073, '/1073/gen-g-vs-triumph-pulse-invitational-groups-group-c'), (1051, '/1051/terror-net-vs-bottom-fraggers-brazil-spray-series-gf'), (1065, '/1065/cloud9-vs-triumph-pulse-invitational-groups-group-c'), (1047, '/1047/notrab-vs-terror-net-brazil-spray-series-upper-bracket-finals'), (1041, '/1041/team-vikings-vs-exult-gg-brazil-spray-series-lr1'), (616, '/616/need-more-dm-vs-partyparrots-absolute-masters-ro16'), (931, '/931/me-n-my-dawgs-vs-big-frames-trovo-challenge-north-america-qf'), (908, '/908/timbers-esports-vs-estral-esports-ggtech-invitational-la-n-gf'), (927, '/927/dpe-vs-varazze-moscow-five-open-tournament-cf'), (863, '/863/team-solomid-vs-t1-t1-x-nerd-street-gamers-valorant-showdown-gf'), (932, '/932/qqq-hasagi-vs-divisionx-gaming-the-1st-championship-playoffs-gf'), (834, '/834/viboras-vs-simplicity-gaming-empire-play-north-africa-invitational-tournament-semifinals'), (784, '/784/pants-down-vs-seal-team-6-australian-esports-network-tournament-gf'), (820, '/820/together-we-are-terrific-vs-prospectsgg-t1-x-nerd-street-gamers-valorant-showdown-group-a'), (826, '/826/gen-g-vs-mixup-t1-x-nerd-street-gamers-valorant-showdown-group-d'), (816, '/816/100-thieves-vs-100-blifted-t1-x-nerd-street-gamers-showdown-group-c'), (734, '/734/spotup-vs-echo-8-t1-x-nerd-street-gamers-valorant-showdown-qualifier-2-gf'), (210, '/210/absolute-legends-blue-vs-kk-valorant-absolute-masters-gs-h'), (659, '/659/team-mixwell-vs-team-ange1-g2-esports-invitational-playoffs-sf'), (676, '/676/absolute-jupiter-vs-nora-rengo-rage-invitational-playoffs-semifinals'), (719, '/719/together-we-are-terrific-vs-prospectsgg-t1-x-nerd-street-gamers-valorant-showdown-qualifier-1-gf'), (656, '/656/team-ex6tenz-vs-team-ange1-g2-esports-invitational-group-stage-group-b'), (204, '/204/goagoagoa-vs-superstars-absolute-masters-gs-f'), (653, '/653/team-taimou-vs-team-ex6tenz-g2-esports-invitational-group-stage-group-b'), (654, '/654/team-izak-vs-team-ange1-g2-esports-invitational-group-stage-group-b'), (185, '/185/ferrari-peek-vs-team-spectralis-absolute-masters-gs-d'), (622, '/622/team-solomid-vs-303-esports-orange-lof-x-dts-invitational-series-playoffs-qf'), (625, '/625/china-nguyen-vs-rugratz-lof-x-dts-invitational-series-playoffs-qf'), (601, '/601/gamelanders-blue-vs-vknata-copa-rakin-1-gf'), (600, '/600/vknata-vs-cage-triggered-copa-rakin-1-lbf'), (488, '/488/myteq-vs-team-zonixx-fragster-valorant-invitational-rr'), (487, '/487/valorando-vs-myteq-fragster-valorant-invitational-rr'), (492, '/492/small-soldiers-vs-god-squad-oce-gg-community-cup-2-gf'), (439, '/439/mad-vs-vision-strikers-wesl-x-toonation-gf'), (521, '/521/gen-g-vs-together-we-are-terrific-pittsburgh-knights-invitational-gf'), (527, '/527/big-frames-vs-china-nguyen-pittsburgh-knights-invitational-lr3'), (524, '/524/big-frames-vs-prospectsgg-pittsburgh-knights-invitational-lr2'), (519, '/519/gen-g-vs-big-frames-pittsburgh-knights-invitational-ubsf'), (479, '/479/big-frames-vs-china-nguyen-pittsburgh-knights-invitational-qf'), (510, '/510/together-we-are-terrific-vs-big-frames-pittsburgh-knights-invitational-groups-c'), (509, '/509/quick-shots-vs-big-frames-pittsburgh-knights-invitational-groups-c'), (507, '/507/big-frames-vs-way-2-french-pittsburgh-knights-invitational-groups-c'), (470, '/470/gen-g-vs-china-nguyen-clg-blitz-cup-2-tournament-qf'), (446, '/446/way-2-french-vs-echo-8-clg-blitz-cup-2-tournament-r16'), (447, '/447/tsm-vs-tbd-clg-blitz-cup-2-tournament-r16'), (449, '/449/imt-vs-tbd-clg-blitz-cup-2-tournament-r16'), (450, '/450/gen-g-vs-tbd-clg-blitz-cup-2-tournament-r16'), (200, '/200/royals-vs-need-more-dm-absolute-masters-gs-e'), (403, '/403/tsm-vs-6k-lof-x-dts-invitational-series-w4-qf'), (226, '/226/team-solomid-vs-t1-twitch-rivals-valorant-launch-showdown-na-gf'), (223, '/223/frn-vs-dyl-twitch-rivals-valorant-launch-showdown-latam-2-gf'), (221, '/221/pan-vs-mch-twitch-rivals-valorant-launch-showdown-gf'), (219, '/219/scy-vs-pan-twitch-rivals-valorant-launch-showdown-sf'), (220, '/220/mch-vs-pn-twitch-rivals-valorant-launch-showdown-sf'), (225, '/225/sab-vs-ken-twitch-rivals-valorant-launch-showdown-gf'), (444, '/444/vs-vs-tbd-wesl-x-toonation-qf'), (443, '/443/egel-vs-tbd-wesl-x-toonation-qf'), (343, '/343/t1-vs-immortals-twitch-rivals-valorant-launch-showdown-na-b'), (345, '/345/tac-vs-tke-twitch-rivals-valorant-launch-showdown-na-b'), (342, '/342/brax-vs-tac-twitch-rivals-valorant-launch-showdown-na-b'), (222, '/222/blu-vs-mym-twitch-rivals-valorant-launch-showdown-latam-1-gf'), (456, '/456/echo-8-vs-tbd-nerd-street-gamers-monthly-1-uqf'), (458, '/458/terrific-vs-tbd-nerd-street-gamers-monthly-1-lor1'), (333, '/333/fabriken-vs-worst-players-twitch-rivals-valorant-launch-showdown-eu-2-sf'), (291, '/291/prodigy-vs-washed-players-twitch-rivals-valorant-launch-showdown-eu-1-qf'), (442, '/442/mad-vs-tbd-wesl-x-toonation-qf'), (218, '/218/jua-vs-stx-twitch-rivals-valorant-launch-showdown-gf'), (438, '/438/lavega-vs-tbd-wesl-x-toonation-qf'), (1004, '/1004/team-turkey-vs-purple-cobras-clutch-battles-lair-gg-gf'), (603, '/603/startedfromcs-vs-blackbelts-lestream-x-bmw-the-2-gran-coup-gf'), (245, '/245/fordon-boars-vs-al-epulze-valorant-prodigies-b'), (246, '/246/fish123-vs-valorando-epulze-valorant-prodigies-group-stage-group-b'), (242, '/242/al-vs-valorando-epulze-valorant-prodigies-b'), (168, '/168/exceptis-vs-need-more-dm-absolute-masters-gs-e'), (233, '/233/dreamchasers-vs-hyp-epulze-valorant-prodigies-a'), (232, '/232/worst-players-vs-dreamchasers-epulze-valorant-prodigies-a'), (166, '/166/spectralis-vs-partyparrots-absolute-masters-gs-d'), (415, '/415/prodigy-vs-123-take-the-throne-3-grand-final'), (157, '/157/fish123-vs-prodigy-solary-cup-playoffs-gf'), (171, '/171/prodigy-vs-avenue-absolute-masters-gs-g'), (156, '/156/hyp-vs-prodigy-solary-cup-playoffs-sf'), (173, '/173/nip-vs-al-blue-absolute-masters-gs-h'), (169, '/169/goa-vs-uruguay-3-absolute-masters-gs-f'), (167, '/167/royals-vs-ameno-absolute-masters-gs-e'), (555, '/555/mousespaz-vs-jacked-gamers-clg-blitz-cup-1-tournament-gf'), (152, '/152/fish123-vs-blackbelts-solary-cup-playoffs-qf'), (153, '/153/hyp-vs-inet-solary-cup-playoffs-qf'), (154, '/154/prodigy-vs-demise-solary-cup-playoffs-qf'), (159, '/159/fordon-boars-vs-clan123-absolute-masters-gs-a'), (416, '/416/prodigy-vs-td-take-the-throne-2-grand-final'), (101, '/101/jacked-gamers-vs-the-boys-valhalla-invitational-w2-playoffs'), (92, '/92/prodigy-vs-shazoo-valorant-academy-eu-closed-series-w3-gf'), (100, '/100/smoke-vs-moscow-5-academy-valhalla-invitational-w2-playoffs'), (82, '/82/nip-vs-2g4l-mandatory-cup-playoffs-bf'), (89, '/89/shazoo-vs-lujaika-valorant-academy-eu-closed-series-w3-sf'), (80, '/80/fish123-vs-2g4l-mandatory-cup-playoffs-sf'), (58, '/58/prodigy-vs-2g4l-fnatic-proving-grounds-valorant-open-tournament-gf'), (49, '/49/royals-vs-tbd-fnatic-proving-grounds-valorant-open-tournament-r16'), (51, '/51/blackbelts-vs-tbd-fnatic-proving-grounds-valorant-open-tournament-r16'), (34, '/34/gen-g-vs-brax-t1-x-nerd-street-gamers-invitational-tournament-gf'), (31, '/31/gen-g-vs-brax-t1-x-nerd-street-gamers-invitational-tournament-uf'), (21, '/21/gen-g-vs-prospects-t1-x-nerd-street-gamers-invitational-tournament-uqf'), (22, '/22/mousespaz-vs-team-shroud-t1-x-nerd-street-gamers-invitational-tournament-uqf'), (23, '/23/sentinels-vs-sonii-t1-x-nerd-street-gamers-invitational-tournament-uqf'), (24, '/24/brax-vs-kurt-t1-x-nerd-street-gamers-invitational-tournament-uqf'), (417, '/417/prodigy-vs-hyp-take-the-throne-1-grand-final'), (20, '/20/smoke-vs-highground-valhalla-invitational-w1-playoffs'), (18, '/18/smoke-vs-whatever-works-valhalla-invitational-w1-playoffs'), (19, '/19/highground-vs-lmh-valhalla-invitational-w1-playoffs'), (187, '/187/fish123-vs-inst-grizicup-playoffs-gf'), (197, '/197/inst-vs-tbd-grizicup-playoffs-sf'), (198, '/198/fish123-vs-tbd-grizicup-playoffs-sf'), (195, '/195/fish123-vs-tbd-grizicup-playoffs-qf'), (196, '/196/inst-vs-tbd-grizicup-playoffs-qf'), (192, '/192/nip-vs-need-more-dm-cooler-cup-playoffs-sf')]

In [211]:
failed_list = []

In [212]:
for idx, combo in enumerate(try_list):
    print(combo[1])
    try:
        url = 'https://www.vlr.gg' + combo[1]
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except:
        try:
            url = 'https://www.vlr.gg' + combo[1]
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
        except:
            failed_list.append(combo)
            continue
            
    
    try:
        match_row = getMatchHeader(soup, combo[0])
        match_df = pd.DataFrame(match_row, index=[0])
        team1_id = match_row['Team1ID']
        team2_id = match_row['Team2ID']
        
        game_dict_list, rbr_dict_list, player_stats_dict_list, game_id_list = getMapStats(soup, url, team1_id, team2_id, combo[0])
    
        if game_id_list is None:
            print('(1) Could not get info for link: ')
            print(combo[1])
            failed_list.append(combo)
            continue
    except:
        print('(2) Could not get info for link: ')
        print(combo[1])
        failed_list.append(combo)
        continue

    adv_player_stats = getPerformanceData(url, game_id_list)
    player_stats_df = pd.DataFrame(player_stats_dict_list)
    adv_player_stats_df = pd.DataFrame(adv_player_stats)

    if adv_player_stats is not None:
        all_player_stats = player_stats_df.merge(adv_player_stats_df, on=['GameID', 'PlayerName', 'Agent'])
        all_player_stats = all_player_stats.drop(columns=['TeamAbbreviation_y'])
    else:
        all_player_stats = player_stats_df
        all_player_stats['Num_2Ks'] = None
        all_player_stats['Num_3Ks'] = None
        all_player_stats['Num_4Ks'] = None
        all_player_stats['Num_5Ks'] = None
        all_player_stats['OnevOne'] = None
        all_player_stats['OnevTwo'] = None
        all_player_stats['OnevThree'] = None
        all_player_stats['OnevFour'] = None
        all_player_stats['OnevFive'] = None
        all_player_stats['Econ'] = None
        all_player_stats['Plants'] = None
        all_player_stats['Defuses'] = None
        all_player_stats['Econ'] = None


    economy_dict_list, economy_rbr_dict_list = getEconomy(url, game_id_list)

    if len(economy_dict_list) > 0:
        game_df = pd.DataFrame(game_dict_list)
        econ_df = pd.DataFrame(economy_dict_list)
        full_game_df = game_df.merge(econ_df, on=['GameID'])

        game_rounds = []
        for key in rbr_dict_list:
            econ_info = economy_rbr_dict_list[key]
            for idx, round_num in enumerate(rbr_dict_list[key]):
                rbr_dict_list[key][round_num].update(econ_info[round_num])

            res = {
                'GameID': key,
                'Team1ID': team1_id,
                'Team2ID': team2_id,
                'RoundHistory': str(rbr_dict_list[key])
            }
            game_rounds.append(res)

        game_rounds_df = pd.DataFrame(game_rounds)
    else:
        full_game_df = pd.DataFrame(game_dict_list)
        game_rounds = []
        for game in game_id_list:
            res = {
                'GameID': game,
                'Team1ID': team1_id,
                'Team2ID': team2_id,
                'RoundHistory': None
            }
            game_rounds.append(res)
        game_rounds_df = pd.DataFrame(game_rounds)


    try:
        match_df.to_sql('Matches', conn, index=False, if_exists='append')
    except:
        print(f'Unable to add to Match Table for {combo[1]}')
        traceback.print_exc()
        pass
    try:
        full_game_df.to_sql('Games', conn, index=False, if_exists='append')
    except:
        print(f'Unable to add to Game Table for {combo[1]}')
        traceback.print_exc()
        pass
    try:
        game_rounds_df.to_sql('Game_Rounds', conn, index=False, if_exists='append')
    except:
        print(f'Unable to add to Game_Rounds Table for {combo[1]}')
        traceback.print_exc()
        pass
    try:
        all_player_stats.to_sql('Game_Scoreboard', conn, index=False, if_exists='append')
    except:
        print(f'Unable to add to Game_Scoreboard Table for {combo[1]}')
        traceback.print_exc()
        pass

/61395/bait-academy-vs-team-reckaroo-knights-monthly-gauntlet-december-ro16
(2) Could not get info for link: 
/61395/bait-academy-vs-team-reckaroo-knights-monthly-gauntlet-december-ro16
/61444/god-particles-vs-reckoning-esports-asus-rog-showdown-series-sf
(2) Could not get info for link: 
/61444/god-particles-vs-reckoning-esports-asus-rog-showdown-series-sf
/60388/vietnamese-curry-vs-froggers-frag-valorant-major-fullerton-qf
(2) Could not get info for link: 
/60388/vietnamese-curry-vs-froggers-frag-valorant-major-fullerton-qf
/60301/booster-seat-gaming-vs-iwnl-frag-valorant-major-philadelphia-r1
(2) Could not get info for link: 
/60301/booster-seat-gaming-vs-iwnl-frag-valorant-major-philadelphia-r1
/55503/ex-flickbaiters-vs-andbox-nerd-street-gamers-winter-championship-open-8-ro16
(2) Could not get info for link: 
/55503/ex-flickbaiters-vs-andbox-nerd-street-gamers-winter-championship-open-8-ro16
/51266/acend-vs-keyd-stars-valorant-champions-opening-a
(2) Could not get info for link: 


(2) Could not get info for link: 
/40469/fox-gaming-vs-geekay-esports-strike-arabia-championship-finals-ubqf
/33853/wildcard-gaming-vs-team-bliss-oceania-tour-championship-ubsf
(2) Could not get info for link: 
/33853/wildcard-gaming-vs-team-bliss-oceania-tour-championship-ubsf
/34813/kafalar-esports-vs-stanbul-wildcats-esa-open-fire-all-stars-playoffs-qf
(2) Could not get info for link: 
/34813/kafalar-esports-vs-stanbul-wildcats-esa-open-fire-all-stars-playoffs-qf
/35238/axibyte-esports-vs-123tap-rix-gg-open-august-sf
(2) Could not get info for link: 
/35238/axibyte-esports-vs-123tap-rix-gg-open-august-sf
/35039/o2-gaming-vs-divine-force-gaming-fgc-valorant-invitational-r4-low
(2) Could not get info for link: 
/35039/o2-gaming-vs-divine-force-gaming-fgc-valorant-invitational-r4-low
/34734/123tap-vs-les-zinzins-de-lespace-lvp-rising-series-3-ro32
(2) Could not get info for link: 
/34734/123tap-vs-les-zinzins-de-lespace-lvp-rising-series-3-ro32
/33320/divize-vs-resonate-nerd-street-gam

(2) Could not get info for link: 
/24978/oxygen-esports-vs-fire-flux-esports-champions-tour-turkey-stage-3-challengers-1-ubqf
/23085/team-bds-vs-megastitut-valorant-open-tour-france-summer-qualifier-gf
(2) Could not get info for link: 
/23085/team-bds-vs-megastitut-valorant-open-tour-france-summer-qualifier-gf
/24023/100-thieves-vs-noble-champions-tour-north-america-stage-3-challengers-1-ro16
(2) Could not get info for link: 
/24023/100-thieves-vs-noble-champions-tour-north-america-stage-3-challengers-1-ro16
/24013/luminosity-vs-complexity-champions-tour-north-america-stage-3-challengers-1-ro32
(2) Could not get info for link: 
/24013/luminosity-vs-complexity-champions-tour-north-america-stage-3-challengers-1-ro32
/23922/darkzero-esports-vs-night-raid-champions-tour-north-america-stage-3-challengers-1-ro128
(2) Could not get info for link: 
/23922/darkzero-esports-vs-night-raid-champions-tour-north-america-stage-3-challengers-1-ro128
/23948/andbox-vs-pirate-nation-champions-tour-north-

(2) Could not get info for link: 
/12077/slick-vs-b4-esports-champions-tour-brazil-stage-1-challengers-3-elimination-b
/11442/equinox-esports-vs-on-the-way-champions-tour-north-america-stage-1-challengers-3-ro32
(2) Could not get info for link: 
/11442/equinox-esports-vs-on-the-way-champions-tour-north-america-stage-1-challengers-3-ro32
/11939/akave-esports-vs-pro-hub-gaming-champions-tour-latam-stage-1-challengers-3-main-event-lan-lbf
(2) Could not get info for link: 
/11939/akave-esports-vs-pro-hub-gaming-champions-tour-latam-stage-1-challengers-3-main-event-lan-lbf
/10258/envy-vs-sentinels-champions-tour-north-america-stage-1-challengers-2-gf
(2) Could not get info for link: 
/10258/envy-vs-sentinels-champions-tour-north-america-stage-1-challengers-2-gf
/13035/tilted-vs-mcteam-lpl-summer-cup-legends-2021-r1
(2) Could not get info for link: 
/13035/tilted-vs-mcteam-lpl-summer-cup-legends-2021-r1
/10455/todak-vs-team-shift-efire-master-league-1-asia-group-stage-group-a
(2) Could not g

(2) Could not get info for link: 
/4005/majstersztyk-vs-entropiq-polish-esport-cup-2020-sf
/3883/andbox-vs-moon-raccoons-renegades-x-nsg-invitational-qual-2-gf
(2) Could not get info for link: 
/3883/andbox-vs-moon-raccoons-renegades-x-nsg-invitational-qual-2-gf
/3794/china-nguyen-vs-mamba-mode-gaming-30bomb-invitational-series-2-tournament-gf
(2) Could not get info for link: 
/3794/china-nguyen-vs-mamba-mode-gaming-30bomb-invitational-series-2-tournament-gf
/3816/sumn-fc-vs-oxygen-esports-epic31-online-gf
(2) Could not get info for link: 
/3816/sumn-fc-vs-oxygen-esports-epic31-online-gf
/3858/beastcoast-vs-spacestation-gaming-renegades-x-nsg-invitational-qual-1-gf
(2) Could not get info for link: 
/3858/beastcoast-vs-spacestation-gaming-renegades-x-nsg-invitational-qual-1-gf
/3118/sumn-fc-vs-zarplata-glx-elite-series-group-d-gf
(2) Could not get info for link: 
/3118/sumn-fc-vs-zarplata-glx-elite-series-group-d-gf
/3112/bbl-esports-vs-nolpenki-glx-elite-series-group-c-gf
(2) Could not

(2) Could not get info for link: 
/1314/way-2-french-vs-echo-8-pax-arena-invitational-tournament-group-b
/1264/morning-light-vs-mamba-mode-gaming-nerd-st-gamers-open-15-tournament-gf
(2) Could not get info for link: 
/1264/morning-light-vs-mamba-mode-gaming-nerd-st-gamers-open-15-tournament-gf
/1147/way-2-french-vs-team-underrated-fight-night-arena-8-gf
(2) Could not get info for link: 
/1147/way-2-french-vs-team-underrated-fight-night-arena-8-gf
/1136/opportunists-vs-nolpenki-weplay-valorant-invitational-qualifier-2-gf
(2) Could not get info for link: 
/1136/opportunists-vs-nolpenki-weplay-valorant-invitational-qualifier-2-gf
/1131/need-more-dm-vs-team-xz-weplay-valorant-invitational-qualifier-1-gf
(2) Could not get info for link: 
/1131/need-more-dm-vs-team-xz-weplay-valorant-invitational-qualifier-1-gf
/1089/g2-esports-vs-prodigy-vitality-european-open-playoffs-gf
(2) Could not get info for link: 
/1089/g2-esports-vs-prodigy-vitality-european-open-playoffs-gf
/1088/prodigy-vs-fabrik

(2) Could not get info for link: 
/470/gen-g-vs-china-nguyen-clg-blitz-cup-2-tournament-qf
/446/way-2-french-vs-echo-8-clg-blitz-cup-2-tournament-r16
(2) Could not get info for link: 
/446/way-2-french-vs-echo-8-clg-blitz-cup-2-tournament-r16
/447/tsm-vs-tbd-clg-blitz-cup-2-tournament-r16
(2) Could not get info for link: 
/447/tsm-vs-tbd-clg-blitz-cup-2-tournament-r16
/449/imt-vs-tbd-clg-blitz-cup-2-tournament-r16
(2) Could not get info for link: 
/449/imt-vs-tbd-clg-blitz-cup-2-tournament-r16
/450/gen-g-vs-tbd-clg-blitz-cup-2-tournament-r16
(2) Could not get info for link: 
/450/gen-g-vs-tbd-clg-blitz-cup-2-tournament-r16
/200/royals-vs-need-more-dm-absolute-masters-gs-e
(2) Could not get info for link: 
/200/royals-vs-need-more-dm-absolute-masters-gs-e
/403/tsm-vs-6k-lof-x-dts-invitational-series-w4-qf
(2) Could not get info for link: 
/403/tsm-vs-6k-lof-x-dts-invitational-series-w4-qf
/226/team-solomid-vs-t1-twitch-rivals-valorant-launch-showdown-na-gf
(2) Could not get info for lin

(2) Could not get info for link: 
/34/gen-g-vs-brax-t1-x-nerd-street-gamers-invitational-tournament-gf
/31/gen-g-vs-brax-t1-x-nerd-street-gamers-invitational-tournament-uf
(2) Could not get info for link: 
/31/gen-g-vs-brax-t1-x-nerd-street-gamers-invitational-tournament-uf
/21/gen-g-vs-prospects-t1-x-nerd-street-gamers-invitational-tournament-uqf
(2) Could not get info for link: 
/21/gen-g-vs-prospects-t1-x-nerd-street-gamers-invitational-tournament-uqf
/22/mousespaz-vs-team-shroud-t1-x-nerd-street-gamers-invitational-tournament-uqf
(2) Could not get info for link: 
/22/mousespaz-vs-team-shroud-t1-x-nerd-street-gamers-invitational-tournament-uqf
/23/sentinels-vs-sonii-t1-x-nerd-street-gamers-invitational-tournament-uqf
(2) Could not get info for link: 
/23/sentinels-vs-sonii-t1-x-nerd-street-gamers-invitational-tournament-uqf
/24/brax-vs-kurt-t1-x-nerd-street-gamers-invitational-tournament-uqf
(2) Could not get info for link: 
/24/brax-vs-kurt-t1-x-nerd-street-gamers-invitational-tour

In [213]:
print(len(failed_list))

348
